## 有名なモデル

In [ ]:
import torch
from torch.nn import functional as F
from torch import nn
from torch import optim
import torchvision
from torchvision import transforms
from torchvision import models
from torchvision.models.vgg import VGG11_Weights
from torch.utils.data import DataLoader, Subset
import utils

### LeNet

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5) # 1 input channel, 6 output channels, 5x5 kernel
        self.conv2 = nn.Conv2d(6, 16, 5) # 6 input channels, 16 output channels, 5x5 kernel
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, X):
        X = F.max_pool2d(F.relu(self.conv1(X)), 2)
        X = F.max_pool2d(F.relu(self.conv2(X)), 2)
        X = self.flatten(X)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = F.relu(self.fc3(X))
        return X

### 既存のモデルを使用

In [ ]:
# モデルと重みをダウンロード
model = models.vgg11(weights=VGG11_Weights.IMAGENET1K_V1)

In [ ]:
# データのロード
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
val_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# データ数が多いので最初の1000件だけ
train_dataset_sub = Subset(train_dataset, range(1000))
val_dataset_sub = Subset(val_dataset, range(500))

In [ ]:
# 最終層の出力を10クラス分類用に変更
model.classifier[-1] = nn.Linear(4096, 10)

In [ ]:
opt = optim.SGD(model.parameters(), lr=0.03)

In [32]:
#utils.learn(model, train_loader, val_loader, opt, F.cross_entropy, 3)

Training:  23%|██▎       | 358/1563 [1:21:08<46:58,  2.34s/it]  